In [29]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from collections import defaultdict
from surprise.model_selection.split import LeaveOneOut
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv("all_10.csv")
df.head()

,user_id,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,useful
0,n_HMBasYWRQxVjts_QyF9w,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Cajun/Creole, Seafood, Steakhouses, Restaurants",4.0,0,2014-03-14 04:56:09,0,7H98TWCrkyRxo8DMfYrWRA,5.0,this place hasnt change a bit in their service...,0
1,tXhscxwFTd-ayLTdPx_2yA,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Cajun/Creole, Seafood, Steakhouses, Restaurants",4.0,0,2012-05-28 15:40:38,1,jTlP9izKAXbF9T91pbP7RA,3.0,"I've eaten here twice, it was better a few yea...",0
2,y4O_c6UUAAtPb3Uk-T4t8A,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Cajun/Creole, Seafood, Steakhouses, Restaurants",4.0,0,2017-08-16 06:41:49,0,WYNusZjgf6BsISk829MQKw,5.0,A great culinary experience from start to fini...,0
3,_0Za7N9v4rcUVxHy107a7Q,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Cajun/Creole, Seafood, Steakhouses, Restaurants",4.0,0,2015-11-18 21:41:50,0,cHsv7KZDouejMVih43qRUQ,5.0,Had Dinner at Delmonico on 11-5-15. Still the...,0
4,aZg019PkWURjSnC7-CH0Bw,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Cajun/Creole, Seafood, Steakhouses, Restaurants",4.0,0,2013-05-29 07:23:34,0,CVkTL2dfXOdtpMeFhtWayw,5.0,My wife took me here for a fabulous dinner bef...,0


In [3]:
df.tail()

,user_id,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,useful
410040,R9Bm86DXed10xMvhJrcDdQ,zzzaIBwimxVej4tY6qFOUQ,Guthrie's,"Southern, Restaurants",3.5,0,2012-12-01 21:13:14,0,HXN_Lo5hF9gqlFgaBdxkCg,1.0,ordered the 7 strip and the chicken box would ...,2
410041,BP652XN-JNMhMWzNgZHSRg,zzzaIBwimxVej4tY6qFOUQ,Guthrie's,"Southern, Restaurants",3.5,0,2012-12-24 02:19:28,0,iW5AiBrTtAGgXe3KqANItg,5.0,Guthrie's is a great choice for people who are...,2
410042,jhaT21VWZVtW2zXPVmzeyg,zzzaIBwimxVej4tY6qFOUQ,Guthrie's,"Southern, Restaurants",3.5,0,2012-11-29 22:13:10,0,1WgXVPm4FuMQl07COCBtyw,4.0,Guthrie's is all about one main thing: chicken...,4
410043,mySDXsBhY3u0yI-G2oVDkA,zzzaIBwimxVej4tY6qFOUQ,Guthrie's,"Southern, Restaurants",3.5,1,2013-02-18 21:37:17,0,ql4lCsHh45lXZrf4F2S3yg,4.0,"I have to say, I do like this place a lot more...",1
410044,-gfC7auUsRN2KqTBzHMBZg,zzzaIBwimxVej4tY6qFOUQ,Guthrie's,"Southern, Restaurants",3.5,0,2013-06-14 23:17:36,1,PBs0VwO7arZqlIba6rAVuA,4.0,This is a familiar place to me. I frequented t...,1


In [4]:
df = df[['user_id','business_id','stars']]

In [5]:
df.head()

,user_id,business_id,stars
0,n_HMBasYWRQxVjts_QyF9w,--9e1ONYQuAa-CB_Rrw7Tw,5.0
1,tXhscxwFTd-ayLTdPx_2yA,--9e1ONYQuAa-CB_Rrw7Tw,3.0
2,y4O_c6UUAAtPb3Uk-T4t8A,--9e1ONYQuAa-CB_Rrw7Tw,5.0
3,_0Za7N9v4rcUVxHy107a7Q,--9e1ONYQuAa-CB_Rrw7Tw,5.0
4,aZg019PkWURjSnC7-CH0Bw,--9e1ONYQuAa-CB_Rrw7Tw,5.0


In [6]:
len(df.user_id.unique())

15801

In [7]:
len(df.business_id.unique())

4218

In [8]:
df = df.dropna()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 410045 entries, 0 to 410044
Data columns (total 3 columns):
user_id        410045 non-null object
business_id    410045 non-null object
stars          410045 non-null float64
dtypes: float64(1), object(2)
memory usage: 12.5+ MB


In [10]:
#df_train = pd.read_csv("train_33_selected.csv")

In [11]:
#df_test = pd.read_csv("test_33_selected.csv")

In [12]:
reader = Reader(rating_scale=(1, 5))
#data_train = Dataset.load_from_df(df_train, reader)
#data_test = Dataset.load_from_df(df_test, reader)
data_all = Dataset.load_from_df(df, reader)

In [13]:
df_all = data_all.build_full_trainset()

In [13]:
#df_trainset_train = data_train.build_full_trainset()
#df_trainset_test = df_trainset_train.build_testset()

In [14]:
#df_testset = data_test.build_full_trainset().build_testset()

In [20]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [56]:
top_n = defaultdict(list)

In [58]:
for uid, iid, true_r, est_mean, _ in predictions:
    top_n[uid].append((iid, est_mean))

In [59]:
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:10]

In [60]:
top_n

defaultdict(list,
            {'AWt-4_RGNqz9u2T8XfOy9g': [('2hSIeCX2cV-chFcBBXrZZA',
               4.805635698861936),
              ('O7UMzd3i-Zk8dMeyY9ZwoA', 4.773910290931745),
              ('QOE93SXgKd8EsaUT-DaXhA', 4.752364669363533),
              ('wWuGa3OOQJro_XGe-GKBXA', 4.706374056537814),
              ('9P23-V64kYz3trn9ecaJJA', 4.662774069870176),
              ('vT812nRctS62PVxUverayw', 4.652901468865566),
              ('tMdgY7Ou4k3bYZZulJCiuQ', 4.610910769277154),
              ('O19VReN1I2TBrJsbXUAIJg', 4.609680761901602),
              ('L1-1P3acJc4gEFvWwjXcNQ', 4.5941634743561135),
              ('vOMDU31gdylrzBhAKC9QbA', 4.587676783667592)],
             'HRP5zLOrhXgLYAlaBt-HgA': [('2hSIeCX2cV-chFcBBXrZZA',
               4.959871977927573),
              ('O7UMzd3i-Zk8dMeyY9ZwoA', 4.9281465699973825),
              ('IhNASEZ3XnBHmuuVnWdIwA', 4.909676019390615),
              ('QOE93SXgKd8EsaUT-DaXhA', 4.906600948429169),
              ('wWuGa3OOQJro_XGe-GKBXA', 4.

# Modeling

In [15]:
newset = df_all.build_anti_testset()

#### BaselineOnly

In [66]:
from surprise import BaselineOnly

bsl_options = {'method': 'sgd',
               'reg': 0.001,
               'learning_rate': .005,
               }

algo_base = BaselineOnly(bsl_options=bsl_options)
algo_base.fit(df_all)

Estimating biases using sgd...


In [13]:
predictions = algo.test(newset)

In [31]:
pred_one = algo.predict(uid= 'AWt-4_RGNqz9u2T8XfOy9g', iid= '2hSIeCX2cV-chFcBBXrZZA')
pred_one

Prediction(uid='AWt-4_RGNqz9u2T8XfOy9g', iid='2hSIeCX2cV-chFcBBXrZZA', r_ui=None, est=4.6351312863623875, details={'was_impossible': False})

In [37]:
pd.DataFrame(pred_one).T

,0,1,2,3,4
0,AWt-4_RGNqz9u2T8XfOy9g,2hSIeCX2cV-chFcBBXrZZA,None,4.63513,{'was_impossible': False}


In [14]:
pred_base = pd.DataFrame(predictions)

In [70]:
df_test = pd.DataFrame(set(df['business_id']) - set(rated_item)).rename(columns={0:"iid"})
df_test['uid'] = 'AWt-4_RGNqz9u2T8XfOy9g'
df_test.head()

,iid,uid
0,ALp--wAIVL4qQfCANn3ffw,AWt-4_RGNqz9u2T8XfOy9g
1,B7XwqO9Lbzq-VtOiRobfCg,AWt-4_RGNqz9u2T8XfOy9g
2,v7xjDbezQ2xw95itvN244Q,AWt-4_RGNqz9u2T8XfOy9g
3,9Jc3W0aR9Xf2gcHI0rEXsw,AWt-4_RGNqz9u2T8XfOy9g
4,E96XW5-Me9nnBIkpg0nNBA,AWt-4_RGNqz9u2T8XfOy9g


In [74]:
def test(x):
    x['est_base']=algo.predict(uid=x['uid'], iid=x['iid'])[3]
    return x

df_test = df_test.apply(test,axis=1)

In [16]:
pred_base.to_csv("pred_base_10.csv", index=False)

In [25]:
pred_base = pd.read_csv("pred_base_10.csv")

In [26]:
pred_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66262103 entries, 0 to 66262102
Data columns (total 5 columns):
uid        object
iid        object
r_ui       float64
est        float64
details    object
dtypes: float64(2), object(3)
memory usage: 2.5+ GB


In [21]:
top_n = get_top_n(predictions, n=10)

In [22]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

AWt-4_RGNqz9u2T8XfOy9g ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ', 'vOMDU31gdylrzBhAKC9QbA']
m2gIW3qe18scLqPw2JP2BQ ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
XqsuWJIwdy0OXYvUX1I2-Q ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
rNjgVKaBh1PlUA6dYB6EUw ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

WTE4gtyCs1eJDSCj2Afo_A ['2hSIeCX2cV-chFcBBXrZZA', '9P23-V64kYz3trn9ecaJJA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'vT812nRctS62PVxUverayw', 'wWuGa3OOQJro_XGe-GKBXA', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
y_IaE3HKJKEsw5Cg62D1RA ['2hSIeCX2cV-chFcBBXrZZA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
H7fr4A3OlKtDr-sNn8BIlw ['2hSIeCX2cV-chFcBBXrZZA', '9P23-V64kYz3trn9ecaJJA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
6ZaL6fBlKx5tb8c9opl7rQ ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

lIM5d1HsNmdkn9LzV8gKNw ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
hzKQOJI8-7TM-BstJpSVCA ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
ylCz0y57-aVG_Y9rGC6xTA ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
3RTesI_MAwct13LWm4rhLw ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9eca

BdRMVROS1MXOHxr-bdZv0g ['2hSIeCX2cV-chFcBBXrZZA', '9P23-V64kYz3trn9ecaJJA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'L1-1P3acJc4gEFvWwjXcNQ', 'vOMDU31gdylrzBhAKC9QbA']
0oCX_noDIESiIHKf7Sly1w ['-1m9o3vGRA8IBPNvNqKLmA', '-C8sSrFqaCxp51pyo-fQLQ', '-RJ216TTIghZshCkUlD1WQ', '-iFvYhgysvjkxckCr42NRw', '0q_BHpxbikVtPRRLRu-U0g', '0xvdC8F0HmFpAFkJk6nXLg', '1CaM8eIvl41l4f3V-V-cAw', '1WBVqmo7tgF0z0eGf4fPhQ', '296PZdxSrtH08EUwCsOKMw', '2B46bRpDh49eDyjXGhL_ZQ']
y6UplUH72gl3eBtG6ACa1w ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
FCGCxiXBrj1p_8cscTeQXg ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

oKhpMYvIntENZ5UzuoZWUg ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
O5VAQ-dbR2Bk0OZLL1L0Ig ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
-R_hd3xKlnUIi_m-IZsY9Q ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
eYNLZkBYHTgGUcmgTVOWBQ ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

In [50]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 8,
               'reg_i': 15
               }

algo_als = BaselineOnly(bsl_options=bsl_options)
algo_als.fit(df_trainset_train)

train_pred = algo_als.test(df_trainset_test)
print("BaselineOnly train biased RMSE", accuracy.rmse(train_pred))
test_pred = algo_als.test(df_testset)
print("BaselineOnly test unbiased RMSE", accuracy.rmse(test_pred))

Estimating biases using als...
RMSE: 1.0665
BaselineOnly train biased RMSE 1.0665105660993361
RMSE: 1.2108
BaselineOnly test unbiased RMSE 1.2107896707675672


#### KNN

In [77]:
from surprise import KNNBaseline
algo_knn = KNNBaseline(k=30, user_based=True)
algo_knn.fit(df_all)

def knn(x):
    x['est_knn']=algo_knn.predict(uid=x['uid'], iid=x['iid'])[3]
    return x

df_pred = df_pred.apply(knn,axis=1)
df_pred.head()

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


,iid,uid,est_base,est_knn
0,ALp--wAIVL4qQfCANn3ffw,AWt-4_RGNqz9u2T8XfOy9g,3.773437,3.741602
1,B7XwqO9Lbzq-VtOiRobfCg,AWt-4_RGNqz9u2T8XfOy9g,3.512693,3.332649
2,v7xjDbezQ2xw95itvN244Q,AWt-4_RGNqz9u2T8XfOy9g,2.852100,3.071767
3,9Jc3W0aR9Xf2gcHI0rEXsw,AWt-4_RGNqz9u2T8XfOy9g,2.841094,1.722346
4,E96XW5-Me9nnBIkpg0nNBA,AWt-4_RGNqz9u2T8XfOy9g,4.056445,3.903679


In [17]:
from surprise import KNNBaseline
algo_knn = KNNBaseline(k=30, user_based=True)
algo_knn.fit(df_all)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [18]:
predictions_knn = algo_knn.test(newset)

In [19]:
pred_knn = pd.DataFrame(predictions_knn)

In [25]:
pred_knn.to_csv("pred_knn.csv", index=False)

In [28]:
pred_knn.head()

,uid,iid,r_ui,est,details
0,n_HMBasYWRQxVjts_QyF9w,-0BxAGlIk5DJAGVkpqBXxg,3.78119,3.662262,"{'was_impossible': False, 'actual_k': 12}"
1,n_HMBasYWRQxVjts_QyF9w,-0RkJ_uIduNLWQrphbADRw,3.78119,3.305432,"{'was_impossible': False, 'actual_k': 8}"
2,n_HMBasYWRQxVjts_QyF9w,-1m9o3vGRA8IBPNvNqKLmA,3.78119,4.199829,"{'was_impossible': False, 'actual_k': 19}"
3,n_HMBasYWRQxVjts_QyF9w,-1vfRrlnNnNJ5boOVghMPA,3.78119,3.826044,"{'was_impossible': False, 'actual_k': 8}"
4,n_HMBasYWRQxVjts_QyF9w,-3zffZUHoY8bQjGfPSoBKQ,3.78119,4.173958,"{'was_impossible': False, 'actual_k': 30}"


In [29]:
pred_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66262103 entries, 0 to 66262102
Data columns (total 5 columns):
uid        object
iid        object
r_ui       float64
est        float64
details    object
dtypes: float64(2), object(3)
memory usage: 2.5+ GB


In [ ]:
train_pred_knn = algo_knn.test(df_trainset_test)
print("KNN train biased RMSE", accuracy.rmse(train_pred_knn))
test_pred_knn = algo_knn.test(df_testset)
print("KNN test unbiased RMSE", accuracy.rmse(test_pred_knn))

In [ ]:
#cross_validate(algo_knn, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)kkk

#### PMF

In [38]:
from surprise import SVD
algo_pmf = SVD(n_factors=12, n_epochs=20, biased=False)
algo_pmf.fit(df_trainset_train)

train_pred_pmf = algo_pmf.test(df_trainset_test)
print("PMF train biased RMSE", accuracy.rmse(train_pred_pmf))
test_pred_pmf = algo_pmf.test(df_testset)
print("PMF test unbiased RMSE", accuracy.rmse(test_pred_pmf))

RMSE: 0.9462
PMF train biased RMSE 0.9462048849220677
RMSE: 1.5357
PMF test unbiased RMSE 1.5356683095858437


In [39]:
cross_validate(algo_pmf, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3664  1.3728  1.3646  1.3655  1.3725  1.3684  0.0036  
MAE (testset)     1.0549  1.0607  1.0534  1.0550  1.0594  1.0567  0.0028  
Fit time          15.57   15.76   15.75   15.73   15.74   15.71   0.07    
Test time         1.30    1.28    0.89    0.86    0.85    1.04    0.21    


{'fit_time': (15.566722869873047,
  15.762358903884888,
  15.749221086502075,
  15.731513023376465,
  15.742002964019775),
 'test_mae': array([1.05493793, 1.06067975, 1.05337667, 1.05501464, 1.05939546]),
 'test_rmse': array([1.36638166, 1.37284885, 1.36459553, 1.3655026 , 1.3724832 ]),
 'test_time': (1.299198865890503,
  1.2790918350219727,
  0.8946449756622314,
  0.8617708683013916,
  0.8522119522094727)}

In [26]:
from surprise import SVD
algo_pmf = SVD(biased=False)

for trainset, testset in loo.split(data_all):
    
    algo_pmf.fit(trainset)
    predictions = algo_pmf.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.5246
RMSE: 1.5259
RMSE: 1.5192
RMSE: 1.5190
RMSE: 1.5192


#### SVD

In [78]:
from surprise import SVD
algo_svd = SVD(n_factors=20, n_epochs=15)
algo_svd.fit(df_all)

def svd(x):
    x['est_']=algo_svd.predict(uid=x['uid'], iid=x['iid'])[3]
    return x

df_pred = df_pred.apply(svd,axis=1)
df_pred.head()

,iid,uid,est_base,est_knn
0,ALp--wAIVL4qQfCANn3ffw,AWt-4_RGNqz9u2T8XfOy9g,3.773437,3.756842
1,B7XwqO9Lbzq-VtOiRobfCg,AWt-4_RGNqz9u2T8XfOy9g,3.512693,3.525242
2,v7xjDbezQ2xw95itvN244Q,AWt-4_RGNqz9u2T8XfOy9g,2.852100,2.871458
3,9Jc3W0aR9Xf2gcHI0rEXsw,AWt-4_RGNqz9u2T8XfOy9g,2.841094,3.029010
4,E96XW5-Me9nnBIkpg0nNBA,AWt-4_RGNqz9u2T8XfOy9g,4.056445,4.021963


In [13]:
from surprise import SVD
algo_svd_1 = SVD(n_factors=20, n_epochs=15)
algo_svd_1.fit(df_all)

In [16]:
predictions_svd_1 = algo_svd_1.test(newset)

In [17]:
pred_svd_1 = pd.DataFrame(predictions_svd_1)

In [20]:
pred_svd_1.head()

,uid,iid,r_ui,est,details
0,n_HMBasYWRQxVjts_QyF9w,-0BxAGlIk5DJAGVkpqBXxg,3.78119,3.889714,{'was_impossible': False}
1,n_HMBasYWRQxVjts_QyF9w,-0RkJ_uIduNLWQrphbADRw,3.78119,3.974348,{'was_impossible': False}
2,n_HMBasYWRQxVjts_QyF9w,-1m9o3vGRA8IBPNvNqKLmA,3.78119,4.606223,{'was_impossible': False}
3,n_HMBasYWRQxVjts_QyF9w,-1vfRrlnNnNJ5boOVghMPA,3.78119,3.444178,{'was_impossible': False}
4,n_HMBasYWRQxVjts_QyF9w,-3zffZUHoY8bQjGfPSoBKQ,3.78119,4.422502,{'was_impossible': False}


In [22]:
pred_svd_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66262103 entries, 0 to 66262102
Data columns (total 5 columns):
uid        object
iid        object
r_ui       float64
est        float64
details    object
dtypes: float64(2), object(3)
memory usage: 2.5+ GB


In [23]:
pred_svd_1.to_csv("pred_svd_1_10.csv", index=False)

In [34]:
train_pred_svd_1 = algo_svd_1.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_1))
test_pred_svd_1 = algo_svd_1.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_1))

RMSE: 1.0249
SVD train biased RMSE 1.024856211461352
RMSE: 1.2118
SVD test unbiased RMSE 1.2117590546524613


In [23]:
from surprise import SVD
algo_svd_2 = SVD(n_factors=50, n_epochs=10)
algo_svd_2.fit(df_trainset_train)

train_pred_svd_2 = algo_svd_2.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_2))
test_pred_svd_2 = algo_svd_2.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_2))

RMSE: 1.0388
SVD train biased RMSE 1.0387571058602665
RMSE: 1.2199
SVD test unbiased RMSE 1.2199389887459124


In [28]:
from surprise import SVD
algo_svd_3 = SVD(n_factors=30, n_epochs=10)
algo_svd_3.fit(df_trainset_train)

train_pred_svd_3 = algo_svd_3.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_3))
test_pred_svd_3 = algo_svd_3.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_3))

RMSE: 1.0608
SVD train biased RMSE 1.0608437830508364
RMSE: 1.2187
SVD test unbiased RMSE 1.2186500127989746


In [30]:
from surprise import SVD
algo_svd_4 = SVD(n_factors=30, n_epochs=15)
algo_svd_4.fit(df_trainset_train)

train_pred_svd_4 = algo_svd_4.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_4))
test_pred_svd_4 = algo_svd_4.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_4))

RMSE: 0.9611
SVD train biased RMSE 0.9610708138026886
RMSE: 1.2151
SVD test unbiased RMSE 1.2150919274110736


In [35]:
cross_validate(algo_svd_1, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1482  1.1487  1.1477  1.1494  1.1472  1.1482  0.0008  
MAE (testset)     0.9133  0.9121  0.9130  0.9142  0.9124  0.9130  0.0007  
Fit time          14.69   16.53   15.14   14.76   15.44   15.31   0.67    
Test time         1.25    1.12    1.15    1.12    1.12    1.15    0.05    


{'fit_time': (14.690735816955566,
  16.53309202194214,
  15.13542103767395,
  14.755640983581543,
  15.436825037002563),
 'test_mae': array([0.91330799, 0.91208184, 0.91299981, 0.91418933, 0.91238658]),
 'test_rmse': array([1.14818215, 1.1487161 , 1.14769818, 1.14935646, 1.14718293]),
 'test_time': (1.246934175491333,
  1.125,
  1.1499409675598145,
  1.116776943206787,
  1.117764949798584)}

In [ ]:
algo_svd = SVD()

for trainset, testset in loo.split(data_all):
    
    algo_svd.fit(trainset)
    predictions = algo_svd.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

#### SVDpp

In [56]:
from surprise import SVDpp
algo_svdpp_1 = SVDpp(n_factors=10, n_epochs=10)
algo_svdpp_1.fit(df_trainset_train)

train_pred_svdpp_1 = algo_svdpp_1.test(df_trainset_test)
print("SVDpp train biased RMSE", accuracy.rmse(train_pred_svdpp_1))
test_pred_svdpp_1 = algo_svdpp_1.test(df_testset)
print("SVDpp test unbiased RMSE", accuracy.rmse(test_pred_svdpp_1))

RMSE: 1.0514
SVDpp train biased RMSE 1.051354400669383
RMSE: 1.2118
SVDpp test unbiased RMSE 1.2117677323392348


In [58]:
from surprise import SVDpp
algo_svdpp_2 = SVDpp(n_factors=5, n_epochs=5)
algo_svdpp_2.fit(df_trainset_train)

train_pred_svdpp_2 = algo_svdpp_2.test(df_trainset_test)
print("SVDpp train biased RMSE", accuracy.rmse(train_pred_svdpp_2))
test_pred_svdpp_2 = algo_svdpp_2.test(df_testset)
print("SVDpp test unbiased RMSE", accuracy.rmse(test_pred_svdpp_2))

RMSE: 1.1077
SVDpp train biased RMSE 1.1076843252153576
RMSE: 1.2254
SVDpp test unbiased RMSE 1.2254464718146858


In [60]:
from surprise import SVDpp
algo_svdpp = SVDpp()
algo_svdpp.fit(df_trainset_train)

In [61]:
train_pred_svdpp = algo_svdpp.test(df_trainset_test)
print("SVDpp train biased RMSE", accuracy.rmse(train_pred_svdpp))
test_pred_svdpp = algo_svdpp.test(df_testset)
print("SVDpp test unbiased RMSE", accuracy.rmse(test_pred_svdpp))

RMSE: 0.8178
SVDpp train biased RMSE 0.8177596675789183
RMSE: 1.2224
SVDpp test unbiased RMSE 1.222432032057506


In [59]:
cross_validate(algo_svdpp_1, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1457  1.1449  1.1483  1.1488  1.1483  1.1472  0.0016  
MAE (testset)     0.9121  0.9110  0.9132  0.9139  0.9136  0.9128  0.0011  
Fit time          137.76  154.74  164.27  160.81  159.68  155.45  9.36    
Test time         6.79    8.01    7.91    10.56   7.72    8.20    1.26    


{'fit_time': (137.75702691078186,
  154.73514199256897,
  164.268061876297,
  160.80781507492065,
  159.68211698532104),
 'test_mae': array([0.91207069, 0.91103457, 0.91324639, 0.91389616, 0.91358074]),
 'test_rmse': array([1.14566271, 1.14489693, 1.14832405, 1.14878058, 1.1483142 ]),
 'test_time': (6.7922258377075195,
  8.010616779327393,
  7.907413005828857,
  10.5589120388031,
  7.715319871902466)}

#### NMF

In [54]:
from surprise import NMF

algo_nmf_1 = NMF(n_factors=500,n_epochs=10,biased=True)
algo_nmf_1.fit(df_trainset_train)

train_pred_nmf_1 = algo_nmf_1.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf_1))
test_pred_nmf_1 = algo_nmf_1.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf_1))

RMSE: 1.8887
NMF train biased RMSE 1.8886954324179221
RMSE: 2.0244
NMF test unbiased RMSE 2.024350988002533


In [53]:
from surprise import NMF

algo_nmf_2 = NMF(n_factors=3,n_epochs=5,biased=True)
algo_nmf_2.fit(df_trainset_train)

train_pred_nmf_2 = algo_nmf_2.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf_2))
test_pred_nmf_2 = algo_nmf_2.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf_2))

RMSE: 1.0423
NMF train biased RMSE 1.0422730231469668
RMSE: 1.2333
NMF test unbiased RMSE 1.2332936708221804


In [29]:
from surprise import NMF

algo_nmf_3 = NMF(n_factors=3,n_epochs=5,biased=True)
algo_nmf_3.fit(df_all)

In [30]:
predictions_nmf_3 = algo_nmf_3.test(newset)

In [31]:
pred_nmf_3 = pd.DataFrame(predictions_nmf_3)

In [32]:
pred_nmf_3.to_csv("pred_nmf_3_10.csv",index=False)

In [33]:
pred_nmf_3.head()

,uid,iid,r_ui,est,details
0,n_HMBasYWRQxVjts_QyF9w,-0BxAGlIk5DJAGVkpqBXxg,3.78119,4.171456,{'was_impossible': False}
1,n_HMBasYWRQxVjts_QyF9w,-0RkJ_uIduNLWQrphbADRw,3.78119,4.219420,{'was_impossible': False}
2,n_HMBasYWRQxVjts_QyF9w,-1m9o3vGRA8IBPNvNqKLmA,3.78119,4.680637,{'was_impossible': False}
3,n_HMBasYWRQxVjts_QyF9w,-1vfRrlnNnNJ5boOVghMPA,3.78119,3.681367,{'was_impossible': False}
4,n_HMBasYWRQxVjts_QyF9w,-3zffZUHoY8bQjGfPSoBKQ,3.78119,4.465805,{'was_impossible': False}


In [ ]:
train_pred_nmf_3 = algo_nmf_3.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf_3))
test_pred_nmf_3 = algo_nmf_3.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf_3))

In [58]:
from surprise import NMF

algo_nmf = NMF()
algo_nmf.fit(df_trainset_train)

In [59]:
train_pred_nmf = algo_nmf.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf))
test_pred_nmf = algo_nmf.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf))

RMSE: 0.6135
NMF train biased RMSE 0.6135013087584554
RMSE: 1.3930
NMF test unbiased RMSE 1.3929995422178636


In [55]:
cross_validate(algo_nmf_2, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1725  1.1729  1.1715  1.1734  1.1682  1.1717  0.0018  
MAE (testset)     0.9237  0.9249  0.9243  0.9249  0.9206  0.9237  0.0016  
Fit time          4.38    6.95    8.13    6.42    10.26   7.23    1.94    
Test time         1.48    1.75    1.68    2.13    1.73    1.75    0.21    


{'fit_time': (4.378977060317993,
  6.947607040405273,
  8.1286461353302,
  6.4235310554504395,
  10.257351160049438),
 'test_mae': array([0.92372323, 0.92493659, 0.92429811, 0.92490481, 0.92059365]),
 'test_rmse': array([1.17247172, 1.17287832, 1.17151232, 1.17341319, 1.16824885]),
 'test_time': (1.4765958786010742,
  1.7464051246643066,
  1.6839540004730225,
  2.1256210803985596,
  1.7344229221343994)}

In [29]:
from surprise import NMF

algo_nmf = NMF()

for trainset, testset in loo.split(data_all):
    
    algo_nmf.fit(trainset)
    predictions = algo_nmf.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.3859
RMSE: 1.3855
RMSE: 1.3893
RMSE: 1.3844
RMSE: 1.3880


#### CoClustering

In [44]:
from surprise import CoClustering

algo_clu_2 = CoClustering(n_cltr_u=3, n_cltr_i=2, n_epochs=5)
algo_clu_2.fit(df_trainset_train)

train_pred_clu_2 = algo_clu_2.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu_2))
test_pred_clu_2 = algo_clu_2.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu_2))

RMSE: 0.9696
CoClustering train biased RMSE 0.969627115190611
RMSE: 1.3367
CoClustering test unbiased RMSE 1.3367309507289586


In [45]:
from surprise import CoClustering

algo_clu_1 = CoClustering(n_cltr_u=2, n_cltr_i=2, n_epochs=5)
algo_clu_1.fit(df_trainset_train)

train_pred_clu_1 = algo_clu_1.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu_1))
test_pred_clu_1 = algo_clu_1.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu_1))

RMSE: 0.9823
CoClustering train biased RMSE 0.9822603928968215
RMSE: 1.3331
CoClustering test unbiased RMSE 1.3330527088008828


In [14]:
from surprise import CoClustering

algo_clu_3 = CoClustering(n_cltr_u=2, n_cltr_i=3, n_epochs=10)
algo_clu_3.fit(df_all)

In [16]:
predictions_clu_3 = algo_clu_3.test(newset)

In [17]:
pred_clu_3 = pd.DataFrame(predictions_clu_3)

In [18]:
pred_clu_3.to_csv("pred_clu_3_10.csv",index=False)

In [19]:
pred_clu_3.head()

,uid,iid,r_ui,est,details
0,n_HMBasYWRQxVjts_QyF9w,-0BxAGlIk5DJAGVkpqBXxg,3.78119,3.656606,{'was_impossible': False}
1,n_HMBasYWRQxVjts_QyF9w,-0RkJ_uIduNLWQrphbADRw,3.78119,3.837659,{'was_impossible': False}
2,n_HMBasYWRQxVjts_QyF9w,-1m9o3vGRA8IBPNvNqKLmA,3.78119,4.617496,{'was_impossible': False}
3,n_HMBasYWRQxVjts_QyF9w,-1vfRrlnNnNJ5boOVghMPA,3.78119,3.565147,{'was_impossible': False}
4,n_HMBasYWRQxVjts_QyF9w,-3zffZUHoY8bQjGfPSoBKQ,3.78119,4.573937,{'was_impossible': False}


In [ ]:
train_pred_clu_3 = algo_clu_3.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu_3))
test_pred_clu_3 = algo_clu_3.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu_3))

In [55]:
from surprise import CoClustering

algo_clu = CoClustering()
algo_clu.fit(df_trainset_train)

In [56]:
train_pred_clu = algo_clu.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu))
test_pred_clu = algo_clu.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu))

RMSE: 0.9677
CoClustering train biased RMSE 0.9677336945010659
RMSE: 1.3402
CoClustering test unbiased RMSE 1.3401557418150414


In [47]:
cross_validate(algo_clu_3, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2743  1.2712  1.2698  1.2734  1.2712  1.2720  0.0016  
MAE (testset)     0.9498  0.9486  0.9482  0.9507  0.9490  0.9492  0.0009  
Fit time          15.92   15.75   21.61   15.73   13.82   16.57   2.64    
Test time         3.08    1.01    2.13    0.90    0.89    1.60    0.88    


{'fit_time': (15.916110038757324,
  15.751596212387085,
  21.609081983566284,
  15.73489499092102,
  13.816364049911499),
 'test_mae': array([0.94981774, 0.9485647 , 0.9481505 , 0.95067871, 0.94895787]),
 'test_rmse': array([1.27425011, 1.27124372, 1.26976499, 1.27342349, 1.27121491]),
 'test_time': (3.0831899642944336,
  1.0070009231567383,
  2.1267809867858887,
  0.8989031314849854,
  0.8862819671630859)}

In [28]:
from surprise import CoClustering

algo_clu = CoClustering()

for trainset, testset in loo.split(data_all):
    
    algo_clu.fit(trainset)
    predictions = algo_clu.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.3439
RMSE: 1.3483
RMSE: 1.3454
RMSE: 1.3469
RMSE: 1.3460


#### Ensemble

In [21]:
pred_base = pd.read_csv("pred_base_10.csv")
pred_knn = pd.read_csv("pred_knn_10.csv")
pred_nmf = pd.read_csv("pred_nmf_3_10.csv")
pred_svd = pd.read_csv("pred_svd_1_10.csv")

test = pred_base

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66262103 entries, 0 to 66262102
Data columns (total 5 columns):
uid        object
iid        object
r_ui       float64
est        float64
details    object
dtypes: float64(2), object(3)
memory usage: 2.5+ GB


In [23]:
test['est_cluster'] = pd.DataFrame(pred_clu_3)['est']
test['est_svd'] = pd.DataFrame(pred_svd)['est']
#test['est_svdpp'] = pd.DataFrame(pred_svdpp_1)['est']
test['est_nmf'] = pd.DataFrame(pred_nmf)['est']
test['est_knn'] = pd.DataFrame(pred_knn)['est']

In [24]:
test['est_mean'] = test[['est','est_cluster','est_svd','est_knn','est_nmf']].mean(axis=1)

In [27]:
test.to_csv("pred_10_est.csv",index=False)

In [28]:
test.head()

,uid,iid,r_ui,est,details,est_cluster,est_svd,est_nmf,est_knn,est_mean
0,n_HMBasYWRQxVjts_QyF9w,-0BxAGlIk5DJAGVkpqBXxg,3.78119,3.927335,{'was_impossible': False},3.656606,3.889714,4.171456,3.662262,3.861474
1,n_HMBasYWRQxVjts_QyF9w,-0RkJ_uIduNLWQrphbADRw,3.78119,4.016495,{'was_impossible': False},3.837659,3.974348,4.219420,3.305432,3.870671
2,n_HMBasYWRQxVjts_QyF9w,-1m9o3vGRA8IBPNvNqKLmA,3.78119,4.637529,{'was_impossible': False},4.617496,4.606223,4.680637,4.199829,4.548343
3,n_HMBasYWRQxVjts_QyF9w,-1vfRrlnNnNJ5boOVghMPA,3.78119,3.470106,{'was_impossible': False},3.565147,3.444178,3.681367,3.826044,3.597368
4,n_HMBasYWRQxVjts_QyF9w,-3zffZUHoY8bQjGfPSoBKQ,3.78119,4.469094,{'was_impossible': False},4.573937,4.422502,4.465805,4.173958,4.421059


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66262103 entries, 0 to 66262102
Data columns (total 10 columns):
uid            object
iid            object
r_ui           float64
est            float64
details        object
est_cluster    float64
est_svd        float64
est_nmf        float64
est_knn        float64
est_mean       float64
dtypes: float64(7), object(3)
memory usage: 4.9+ GB


In [64]:
train = pd.DataFrame(train_pred)

In [67]:
train.to_csv("train_est.csv")

In [65]:
train['est_cluster'] = pd.DataFrame(train_pred_clu_3)['est']
train['est_svd'] = pd.DataFrame(train_pred_svd_1)['est']
train['est_svdpp'] = pd.DataFrame(train_pred_svdpp_1)['est']
train['est_nmf'] = pd.DataFrame(train_pred_nmf_2)['est']
#train['est_knn'] = pd.DataFrame(train_pred_knn)['est']

In [66]:
train['est_mean'] = train[['est','est_cluster','est_svd','est_svdpp','est_nmf']].mean(axis=1)

In [69]:
from sklearn.metrics import mean_squared_error
from math import sqrt

train_rmse = sqrt(mean_squared_error(train['r_ui'], train['est_mean']))
test_rmse = sqrt(mean_squared_error(test['r_ui'], test['est_mean']))

In [70]:
print("ensemble train rmse", train_rmse)
print("ensemble test rmse", test_rmse)

ensemble train rmse 1.0115591563326694
ensemble test rmse 1.2078211477272622


### Blender

In [3]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def train_test_model_performance(clf, X_train, y_train, X_test, y_test):
    # Fit a model by providing X and y from training set
    clf.fit(X_train, y_train)

    # Make prediction on the training data
    y_train_pred = clf.predict(X_train)

    # Make predictions on test data
    y_test_pred = clf.predict(X_test)
    
    train_rmse = sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = sqrt(mean_squared_error(y_test, y_test_pred))

    return train_rmse, test_rmse

In [83]:
#X_train = train[['est','est_cluster','est_svd','est_svdpp','est_nmf']]
#y_train = train['r_ui']
from sklearn.model_selection import train_test_split

X = test[['est','est_cluster','est_svd','est_svdpp','est_nmf']]
y = test['r_ui']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [84]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

train_rmse, test_rmse = train_test_model_performance(reg, X_train, y_train, X_test, y_test)

print('train rmse', train_rmse)
print('test rmse', test_rmse)

train rmse 1.2040537215312836
test rmse 1.2046226769340245


In [87]:
from sklearn.linear_model import Ridge

rig = Ridge(alpha=10)

train_rmse, test_rmse = train_test_model_performance(rig, X_train, y_train, X_test, y_test)

print('train rmse', train_rmse)
print('test rmse', test_rmse)

train rmse 1.2040540469888974
test rmse 1.2046244824045316


In [86]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=20, max_depth=7, min_samples_leaf=10, max_features=3)

train_rmse, test_rmse = train_test_model_performance(rf, X_train, y_train, X_test, y_test)

print('train rmse', train_rmse)
print('test rmse', test_rmse)

train rmse 1.1945114072313194
test rmse 1.2022756054693744
